# Gathering fMRI Data
Both anatomical and functional

In [1]:
from notebook_viewer_functions import *
from functions import *
from scivol import *
import numpy as np
import json
import ants
import gzip

In [2]:
proj_root = parent_directory()
print(f"project root: {proj_root}")
t1_input_filepath = os.path.join(proj_root, "media/sub-01/anat/sub-01_T1w.nii.gz")
bold_stim_filepath = os.path.join(proj_root, "media/sub-01/func/sub-01_task-emotionalfaces_run-1_bold.nii.gz")
bold_rest_filepath = os.path.join(proj_root, "media/sub-01/func/sub-01_task-rest_bold.nii.gz")
mni_anat_filepath =  os.path.join(proj_root, "templates/mni_icbm152_t1_tal_nlin_sym_09a.nii")
mni_mask_filepath = os.path.join(proj_root, "templates/mni_icbm152_t1_tal_nlin_sym_09a_mask.nii")

project root: /Users/joachimpfefferkorn/repos/neurovolume


In [ ]:
raw_t1_img = ants.image_read(t1_input_filepath)
raw_stim_bold = ants.image_read(bold_stim_filepath)
raw_rest_bold_img = ants.image_read(bold_rest_filepath)
mni_img = ants.image_read(mni_anat_filepath)
mni_mask_img = ants.image_read(mni_mask_filepath)

In [4]:
import matplotlib.pyplot as plt
from ipywidgets import interact
import numpy as np

In [5]:
print(raw_stim_bold)

ANTsImage
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (64, 64, 35, 185)
	 Spacing    : (4.0, 4.0, 4.0, 2.0)
	 Origin     : (-127.953, 108.933, -74.8393, 0.0)
	 Direction  : [ 1.  0.  0.  0.  0. -1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.]



# Visualize BOLD

In [6]:
# TODO once integrated add masking
default_cmap = 'nipy_spectral'
default_figsize = (4,4)

def explore_4D_vol(vol: np.ndarray, dim="x"):
    def x_coord(slice, time):
        plt.figure(figsize=default_figsize)
        plt.imshow(vol[slice,:,:,time], cmap=default_cmap)
    def y_coord(slice, time):
        plt.figure(figsize=default_figsize)
        plt.imshow(vol[:,slice,:,time], cmap=default_cmap)    
    def z_coord(slice, time):
        plt.figure(figsize=default_figsize)
        plt.imshow(vol[:,:,slice,time], cmap=default_cmap)
    match dim:
        case "x":
            interact(x_coord, slice=(0, vol.shape[0]-1), time=(0, (vol.shape[3]-1)))
        case "y":
            interact(y_coord, slice=(0, vol.shape[1]-1), time=(0, (vol.shape[3]-1)))
        case "z":
            interact(z_coord, slice=(0, vol.shape[2]-1), time=(0, (vol.shape[3]-1)))

In [7]:
print(raw_stim_bold.numpy().shape[3]-1)

184


In [ ]:
explore_4D_vol(raw_stim_bold.numpy(), dim="x")
explore_4D_vol(raw_stim_bold.numpy(), dim="y")
explore_4D_vol(raw_stim_bold.numpy(), dim="z")

interactive(children=(IntSlider(value=31, description='slice', max=63), IntSlider(value=92, description='time'…

interactive(children=(IntSlider(value=31, description='slice', max=63), IntSlider(value=92, description='time'…

interactive(children=(IntSlider(value=17, description='slice', max=34), IntSlider(value=92, description='time'…

*Is it really only 184 time slices? whats the frame rate here?*

# Subtract Baseline from Stimulus to Isolate the Activations

So it looks like this experiment doesn't have a neutral stimulus, just a "rest" . This isn't great, I'd much prefer a Block Design. There are a couple of ways we could build an experiment. One thought I had would be to take the mean of the rest state and then read the activations as being against that? *Is this valid?*

In [9]:
temporal_average_rest = np.mean(raw_rest_bold_img.numpy(), axis=3)

In [10]:
explore_3D_vol(temporal_average_rest)
print(temporal_average_rest.shape)

interactive(children=(IntSlider(value=31, description='slice', max=63), Output()), _dom_classes=('widget-inter…

(64, 64, 35)


In [11]:
def subtract_neutral_3D(experimental, neutral):
    """
    Subtracts a 3D neutral from each time slice in a 4D experimental volume
    For use in experiments which lack a block design and for which the 
    neutral stimulus has been derived from an averaged rest state
    """
    result = np.empty_like(experimental)
    for time_slice in range(experimental.shape[3]):
        result[:,:,:,time_slice] = experimental[:,:,:, time_slice] - neutral
    return result

In [12]:
isolated_BOLD = subtract_neutral_3D(raw_stim_bold.numpy(), temporal_average_rest)

In [13]:
explore_4D_vol(isolated_BOLD, dim="y")

interactive(children=(IntSlider(value=31, description='slice', max=63), IntSlider(value=92, description='time'…

# Measure Change in Volume
Here's another possible approach. Again, I do not know the scientific validity of this. Here we'll take each frame and measure the difference in the bold response from the previous frame.

Grabbing the absolute value between the current and previous timestamp creates the most convenient isolation for our visualization tool. However, this does not mean that it is the most scientifically valuable.

In [14]:
def measure_BOLD_movement(bold_vol, baseline_vol):
    """
    Each frame shows the difference between it and the previous frame. First frame is initialized at zero. 
    """
    result = np.empty_like(bold_vol)
    for time_slice in range(1, bold_vol.shape[3]):
        result[:,:,:,time_slice] = np.absolute(bold_vol[:,:,:,time_slice] - bold_vol[:,:,:,time_slice - 1])
    return result

In [15]:
bold_movement = measure_BOLD_movement(raw_stim_bold.numpy(), temporal_average_rest)

In [16]:
explore_4D_vol(bold_movement)

interactive(children=(IntSlider(value=31, description='slice', max=63), IntSlider(value=92, description='time'…

# Register and Mask BOLD and Anat

We're going to go wit the `bold_movement` volume. Not sure if it's the best, but it's the most convenient to visualize in this context. Or maybe we mask/register the bold first? Is that the best way of going about it?

We're also going to need to account for motion correction and size differences between anat and bold. Oh boy

In [17]:
mni_template = ants.image_read(mni_anat_filepath)
mni_mask = ants.image_read(mni_mask_filepath)

In [ ]:
#Slicing methodology check
sliced_bold = ants.from_numpy(raw_stim_bold.numpy()[:,:,:,50])
sliced_bold02 = ants.from_numpy(raw_stim_bold.numpy()[:,:,:,100])
explore_3D_vol(sliced_bold.numpy())
print(sliced_bold is sliced_bold02)

interactive(children=(IntSlider(value=31, description='slice', max=63), Output()), _dom_classes=('widget-inter…

False


In [ ]:
def bold_masking(bold_img, template, mask, dilate=True):
    indent = "        ➡️"
    print("Masking bold. This might take a while...")
    result = np.empty_like(bold_img.numpy())


    for time_slice in range(bold_img.numpy().shape[3]):
        bold_slice = ants.from_numpy(bold_img.numpy()[:,:,:,time_slice])

        print(f"Masking for time slice {time_slice} out of {bold_img.numpy().shape[3]}")
        print(f"{indent}creating template")
        template_warp_to_bold_anat = ants.registration(
            fixed=bold_slice,
            moving=template, 
            type_of_transform='SyN',
            verbose=False
            )
        
        print(f"{indent}Registering template image")

        print(f"{indent}Creating brain mask")
        brain_mask = ants.apply_transforms(
            fixed=template_warp_to_bold_anat['warpedmovout'],
            moving=mask,
            transformlist=template_warp_to_bold_anat['fwdtransforms'],
            interpolator='nearestNeighbor',
            verbose=False
            )
        if dilate:
            print(f"{indent}Dilating brian mask")
            brain_mask = ants.morphology(brain_mask, radius=4, operation='dilate', mtype='binary')
        print(f"{indent}Applying brain mask and adding to final result")
        result[:,:,:,time_slice] = ants.mask_image(bold_slice, brain_mask).numpy()
    return ants.from_numpy(result)


In [ ]:
masked_bold = bold_masking(raw_stim_bold, mni_template, mni_mask)

Masking bold. This might take a while...
Masking for time slice 0 out of 185
        ➡️creating template
        ➡️Registering template image
        ➡️Creating brain mask
        ➡️Dilating brian mask
        ➡️Applying brain mask and adding to final result
Masking for time slice 1 out of 185
        ➡️creating template
        ➡️Registering template image
        ➡️Creating brain mask
        ➡️Dilating brian mask
        ➡️Applying brain mask and adding to final result
Masking for time slice 2 out of 185
        ➡️creating template
        ➡️Registering template image
        ➡️Creating brain mask
        ➡️Dilating brian mask
        ➡️Applying brain mask and adding to final result
Masking for time slice 3 out of 185
        ➡️creating template
        ➡️Registering template image
        ➡️Creating brain mask
        ➡️Dilating brian mask
        ➡️Applying brain mask and adding to final result
Masking for time slice 4 out of 185
        ➡️creating template
        ➡️Registering t

In [ ]:
explore_4D_vol(masked_bold)
explore_4D_vol(raw_stim_bold.numpy())

interactive(children=(IntSlider(value=31, description='slice', max=63), IntSlider(value=92, description='time'…

interactive(children=(IntSlider(value=31, description='slice', max=63), IntSlider(value=92, description='time'…

Well, it looks like this is the exact same thing

looking at the mri, though, do we even need to mask the brain? Can we just threshold out the purple stuff?